# CSILSP-CS

In [1]:
import gurobipy as gb
from CSILSPGenerator import GeneratorTW_CS

#### Decision variables:

$$
X_t = \begin{cases}
\text{ quantity of product to be produced/supplied in period t }
\end{cases}
$$

$$
I_t = \begin{cases}
\text{ inventory level at the end of period t }
\end{cases}
$$

$$
Y_t = \begin{cases}
1 \text{ if } X_t > 0\\
0 \text{ otherwise }
\end{cases}
$$

#### Parameters:

$$
T = \begin{cases}
\text{ number of periods in the horizon }
\end{cases}
$$

$$
d_t = \begin{cases}
\text{ Customer aggregate demand that must } \\ \text{ be delivered at the end of period t }
\end{cases}
$$

$$
h_t = \begin{cases}
\text{ Holding cost per unit of product in stock at the end of period t }
\end{cases}
$$

$$
p_t = \begin{cases}
\text{ Unit production/supplying cost in period t }
\end{cases}
$$

$$
s_t = \begin{cases}
\text{ Fixed setup cost in period t }
\end{cases}
$$

$$
C_t = \begin{cases}
\text{ Production/Supply capacity at period t }
\end{cases}
$$

#### CSILSP-CS

$$
\begin{alignat}{3}
& \min \sum_{t=1}^{T} (h_tI_t + p_tX_t + s_tY_t) &\\
\text{s.t.} \;\;\;\;\;&\\
I_t &= I_{t-1} + X_t - d_t \;\; , & \forall t \;\;\\
X_t &\leq \sum_{k=t}^{T} d_kY_t \;\; , & \forall t\;\;\\
X_t,I_t &\geq 0 \;\; , & \forall t\;\;\\
X_t & \leq C_tY_t \;\; , & \forall t\;\;\\
\sum_{k=1}^{t}X_k &\leq \sum_{k=1}^{t}\sum_{j=k}^{T}d_{k,j} \;\; , & \forall t\;\;\\
\sum_{k=t_1}^{t_2}X_k &\geq \sum_{k=t_1}^{t_2}\sum_{j=k}^{t_2}d_{k,j} \;\; , & \forall t_1,t_2 (t_1 \leq t_2)\;\;\\
Y_t &\in \{0,1\} \;\; , & \forall t \;\;
\end{alignat}
$$

In [2]:
# generation parameters

if __name__ == '__main__':
    T = 8
    l_b = 10
    u_b = 20

In [3]:
# generate instance

if __name__ == '__main__' or __name__ == '__autoexec__':
    instance = GeneratorTW_CS(T)
    instance.generate(a=l_b,b=u_b,dw_a=l_b,dw_b=u_b)

In [4]:
if __name__ == '__main__':
    print(instance.printData())

       d     C   h   p   s
T                         
0    0.0  97.0  19  16  11
1   20.0  92.0  12  16  11
2   21.0  91.0  18  20  14
3   38.0  92.0  18  11  14
4   58.0  65.0  11  16  17
5   75.0  72.0  14  10  15
6   97.0  62.0  15  16  10
7  103.0  20.0  14  17  11


In [5]:
if __name__ == '__main__':
    print(instance.get_dw())

{(0, 0): 0, (0, 1): 20, (0, 2): 11, (0, 3): 14, (0, 4): 13, (0, 5): 18, (0, 6): 12, (0, 7): 13, (1, 1): 0, (1, 2): 10, (1, 3): 11, (1, 4): 15, (1, 5): 14, (1, 6): 18, (1, 7): 20, (2, 2): 0, (2, 3): 13, (2, 4): 10, (2, 5): 15, (2, 6): 18, (2, 7): 12, (3, 3): 0, (3, 4): 20, (3, 5): 15, (3, 6): 10, (3, 7): 11, (4, 4): 0, (4, 5): 13, (4, 6): 20, (4, 7): 17, (5, 5): 0, (5, 6): 19, (5, 7): 14, (6, 6): 0, (6, 7): 16, (7, 7): 0}


In [6]:
# variable bindings

T = instance.get_T()
d_w = instance.get_dw()
d = instance.get_d()
C = instance.get_C()
h = instance.get_h()
p = instance.get_p()
s = instance.get_s()

In [7]:
# model

csilsp_cs = gb.Model()

Academic license - for non-commercial use only


In [8]:
# decision variables

X = csilsp_cs.addVars(T,vtype=gb.GRB.CONTINUOUS,lb=0.0, name='X')

I = csilsp_cs.addVars(T,vtype=gb.GRB.CONTINUOUS,lb=0.0, name='I')

Y = csilsp_cs.addVars(T,vtype=gb.GRB.BINARY,name='Y')

csilsp_cs.update()
csilsp_cs.write('csilsp_cs.lp')

$$
\begin{alignat}{3}
& \min \sum_{t=1}^{T} (h_tI_t + p_tX_t + s_tY_t) &
\end{alignat}
$$

In [9]:
# objective function

csilsp_cs.setObjective(I.prod(h) + X.prod(p) + Y.prod(s),gb.GRB.MINIMIZE)

csilsp_cs.update()
csilsp_cs.write('csilsp_cs.lp')

$$
\begin{alignat}{3}
I_t &= I_{t-1} + X_t - d_t \;\; , & \forall t
\end{alignat}
$$

In [10]:
# constraints

csilsp_cs.addConstrs( ( I[t] == I[t-1] + X[t] - d[t] for t in range(1,T-1) ),name='constrI')

csilsp_cs.update()
csilsp_cs.write('csilsp_cs.lp')

In [11]:
# Assumption: starting and ending inventories are null

csilsp_cs.addConstr(I[0] == 0,name='startInventoryNull')
csilsp_cs.addConstr(I[T-1] == 0,name='endInventoryNull')

csilsp_cs.update()
csilsp_cs.write('csilsp_cs.lp')

$$
\begin{alignat}{3}
X_t &\leq \sum_{k=t}^{T} d_kY_t \;\; , & \forall t
\end{alignat}
$$

In [12]:
csilsp_cs.addConstrs( ( X[t] <= (gb.quicksum(d[k] for k in range(t,T)))*Y[t] for t in range(T) ),name='constX')

csilsp_cs.update()
csilsp_cs.write('csilsp_cs.lp')

Utilizzare vincolo (6) o vincolo (7):

$$
\begin{alignat}{3}
X_t & \leq C_t \;\; , & \forall t \;\; (6)
\end{alignat}
$$

In [13]:
#csilsp_cs.addConstrs( ( X[t] <= C[t] for t in range(T) ),name='constrC')

#csilsp_cs.update()
#csilsp_cs.write('csilsp_cs.lp')

$$
\begin{alignat}{3}
X_t & \leq C_tY_t \;\; , & \forall t \;\; (7)
\end{alignat}
$$

In [14]:
csilsp_cs.addConstrs( ( X[t] <= C[t]*Y[t] for t in range(T) ),name='constrC')

csilsp_cs.update()
csilsp_cs.write('csilsp_cs.lp')

$$
\begin{alignat}{3}
\sum_{k=1}^{t}X_k &\leq \sum_{k=1}^{t}\sum_{j=k}^{T}d_{k,j} \;\; , & \forall t\;\;
\end{alignat}
$$

In [15]:
csilsp_cs.addConstrs( ( (gb.quicksum(X[k] for k in range(t+1))) <= \
                         (gb.quicksum(d_w[k,j] for k in range(t+1) for j in range(k,T))) \
                         for t in range(T) ),name='constrTimeWindows')

csilsp_cs.update()
csilsp_cs.write('csilsp_cs.lp')

$$
\begin{alignat}{3}
\sum_{k=t_1}^{t_2}X_k &\geq \sum_{k=t_1}^{t_2}\sum_{j=k}^{t_2}d_{k,j} \;\; , & \forall t_1,t_2 (t_1 \leq t_2)\;\;
\end{alignat}
$$

In [16]:
csilsp_cs.addConstrs( ( (gb.quicksum(X[k] for k in range(t1,t2+1))) >= \
                        (gb.quicksum(d_w[k,j] for k in range(t1,t2+1) for j in range(k,t2+1))) \
                        for t2 in range(T) for t1 in range(t2+1) ),name='constrTimeWindowsCS' )

csilsp_cs.update()
csilsp_cs.write('csilsp_cs.lp')

In [17]:
# solve model

csilsp_cs.optimize()

Optimize a model with 68 rows, 24 columns and 208 nonzeros
Variable types: 16 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 4e+02]
Presolve removed 48 rows and 8 columns
Presolve time: 0.00s
Presolved: 20 rows, 16 columns, 71 nonzeros
Variable types: 9 continuous, 7 integer (7 binary)

Root relaxation: objective 7.452031e+03, 12 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7452.03077    0    3          - 7452.03077      -     -    0s
H    0     0                    7474.0000000 7452.03077  0.29%     -    0s
     0     0     cutoff    0      7474.00000 7474.00000  0.00%     -    0s

Explored 1 nodes (14 simplex iterations) in 0.04 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 747

In [18]:
if __name__ == '__main__':
    print("Soluzione ottima:",csilsp_cs.ObjVal)

Soluzione ottima: 7474.0


In [19]:
X

{0: <gurobi.Var X[0] (value 97.0)>,
 1: <gurobi.Var X[1] (value 20.0)>,
 2: <gurobi.Var X[2] (value 21.0)>,
 3: <gurobi.Var X[3] (value 69.0)>,
 4: <gurobi.Var X[4] (value 65.0)>,
 5: <gurobi.Var X[5] (value 72.0)>,
 6: <gurobi.Var X[6] (value 62.0)>,
 7: <gurobi.Var X[7] (value 6.0)>}

In [20]:
Y

{0: <gurobi.Var Y[0] (value 1.0)>,
 1: <gurobi.Var Y[1] (value 1.0)>,
 2: <gurobi.Var Y[2] (value 1.0)>,
 3: <gurobi.Var Y[3] (value 1.0)>,
 4: <gurobi.Var Y[4] (value 1.0)>,
 5: <gurobi.Var Y[5] (value 1.0)>,
 6: <gurobi.Var Y[6] (value 1.0)>,
 7: <gurobi.Var Y[7] (value 1.0)>}

In [21]:
I

{0: <gurobi.Var I[0] (value 0.0)>,
 1: <gurobi.Var I[1] (value 0.0)>,
 2: <gurobi.Var I[2] (value 0.0)>,
 3: <gurobi.Var I[3] (value 31.0)>,
 4: <gurobi.Var I[4] (value 38.0)>,
 5: <gurobi.Var I[5] (value 35.0)>,
 6: <gurobi.Var I[6] (value 0.0)>,
 7: <gurobi.Var I[7] (value 0.0)>}

In [22]:
# execution time
if __name__ == '__main__':
    print("Optimize time: %.5g sec" % csilsp_cs.Runtime)

Optimize time: 0.056963 sec


In [23]:
# Write results to file

results = "----------------------------------------------------------------" + "\n"
results += " Parameter T = %d " % (T) + "\n"
results += " Parameters d,h,p,s,C random generation from %d to %d " % (l_b,u_b) + "\n"
results += " Time window demands (d_w) random generation from %d to %d " % (l_b,u_b) + "\n"
results += " Execution time = %.5g sec " % csilsp_cs.Runtime + "\n"
results += "----------------------------------------------------------------" + "\n\n\n"

In [24]:
with open("csilsp_cs_results.txt", "a") as write_file:
    write_file.write(results)